# Fine-Tuning Round 2: ModernBERT с учетом взвешивания классов 🎯

В данной тетрадке представлен код для второго раунда тонкой настройки модели ModernBERT для задачи многометочной классификации текстов. Основное внимание уделено балансировке классов с использованием взвешенных функций потерь и специальных техник регуляризации ⚖️.

## Ключевые моменты:
- **Взвешивание классов:**  
  Расчет весов для каждого класса с помощью функции `calculate_class_weights` позволяет скорректировать влияние редких меток и учесть дисбаланс в данных 📊.

- **Механизм внимания для меток:**  
  Модуль `LabelAttention` помогает модели учитывать взаимосвязи между различными метками, что улучшает представление скрытых состояний 🧠.

- **Focal Loss:**  
  Реализация `FocalLoss` позволяет фокусироваться на сложных для классификации примерах, снижая влияние легко определяемых классов 🎯.

- **Сглаживание меток (Label Smoothing):**  
  Введение сглаживания меток помогает бороться с переобучением, корректируя значения меток при расчете функции потерь 🤖.

- **Расширенная архитектура:**  
  Комбинация выходов базовой BERT модели и механизма внимания через промежуточные слои (`intermediate`) позволяет достичь более качественных предсказаний 🚀.

- **Процесс обучения:**  
  Код включает подготовку данных, настройку DataLoader-ов, обучение с использованием оптимизатора AdamW, scheduler для управления скоростью обучения, а также реализацию ранней остановки для предотвращения переобучения ⏱️.


In [ ]:
!pip install -q numpy==1.26
!pip install -q packaging
!pip install -q torch==2.1.0 torchvision==0.16.0
!pip install -q setuptools scikit-learn
!pip install --upgrade -q  datasets==3.1.0 accelerate==1.2.1
!pip install -q "git+https://github.com/huggingface/transformers.git@6e0515e99c39444caae39472ee1b2fd76ece32f1" --upgrade

In [ ]:
import transformers

print(f"Transformers version: {transformers.__version__}")

In [1]:
!nvidia-smi

Tue Feb  4 07:24:40 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:81:00.0 Off |                    0 |
| N/A   32C    P0             121W / 700W |  71823MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [1]:
# Стандартные библиотеки Python
from datetime import datetime
import json
import os
from pathlib import Path

# Библиотеки для работы с данными
import numpy as np
import pandas as pd
from sklearn.metrics import (
    classification_report,
    hamming_loss,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score
)
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

# PyTorch и связанные библиотеки
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification
)

# Typing
from typing import Dict, List, Optional, Tuple, Union, Any

# Настройка CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "4"
torch._dynamo.config.disable = True


class MultiLabelTextDataset(Dataset):
    """
    Description:
        Пользовательский класс датасета для многометочной классификации текстов.
        
    Args:
        texts: Список текстов для классификации
        labels: Массив меток для каждого текста
        tokenizer: Токенизатор для предобработки текстов
        max_length: Максимальная длина токенизированного текста
        
    Returns:
        Объект Dataset для использования с DataLoader
        
    Examples:
        >>> dataset = MultiLabelTextDataset(
        ...     texts=['текст1', 'текст2'],
        ...     labels=np.array([[1, 0], [0, 1]]),
        ...     tokenizer=tokenizer,
        ...     max_length=512
        ... )
    """
    
    def __init__(
        self,
        texts: List[str],
        labels: np.ndarray,
        tokenizer: AutoTokenizer,
        max_length: int = 512
    ) -> None:
        # Сохраняем входные данные как атрибуты класса для использования в __getitem__
        self.texts      = texts
        self.labels     = labels
        self.tokenizer  = tokenizer
        self.max_length = max_length

    def __len__(self) -> int:
        # Возвращает количество примеров в датасете (количество текстов)
        return len(self.texts)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Получаем текст и метки по индексу
        text = str(self.texts[idx])
        # Преобразуем метки в float32 для обучения нейронной сети
        labels = self.labels[idx].astype(np.float32)

        # Токенизируем текст
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,     # Добавляем специальные токены ([CLS], [SEP] и т.д.)
            max_length=self.max_length,  # Ограничиваем длину последовательности
            padding='max_length',        # Дополняем последовательность до max_length
            truncation=True,             # Обрезаем тексты, превышающие max_length
            return_tensors='pt'          # Возвращаем тензоры PyTorch вместо списков
        )

        # Возвращаем словарь с входными данными и метками
        return {
            # Преобразуем тензор размерности (1, max_length) в (max_length,)
            'input_ids': encoding['input_ids'].flatten(),
            
            # attention_mask: 1 для настоящих токенов, 0 для padding токенов
            'attention_mask': encoding['attention_mask'].flatten(),
            
            # Преобразуем numpy массив меток в тензор PyTorch
            'labels': torch.FloatTensor(labels)
        }


def load_data(
    data_path: str,
    text_column: str = 'текст'
) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """
    Description:
        Загрузка и предобработка датасета из CSV файла.
        
    Args:
        data_path: Путь к CSV файлу с данными
        text_column: Название колонки с текстами
        
    Returns:
        Кортеж (тексты, метки, названия_меток)
        
    Raises:
        FileNotFoundError: Если файл не найден
    """
    print(f"\nЗагрузка данных из {data_path}...")
    df = pd.read_csv(data_path)
    
    # Разделение признаков и меток
    texts = df[text_column].values
    label_columns = [col for col in df.columns if col != text_column]
    labels = df[label_columns].values
    
    print(f"Загружено {len(texts)} записей с {len(label_columns)} метками")
    return texts, labels, label_columns


def evaluate_multilabel(
    y_true: np.ndarray,
    y_pred: np.ndarray,
    label_columns: Optional[List[str]] = None
) -> Dict[str, Union[float, Dict[str, Dict[str, float]]]]:
    """
    Description:
        Вычисляет метрики для мультилейбл классификации.
        
    Args:
        y_true: Истинные метки
        y_pred: Предсказанные метки
        label_columns: Названия меток для детальной оценки
        
    Returns:
        Словарь с общими метриками и метриками по каждой метке
    """
    # Общие метрики для мультилейбл классификации
    metrics = {
        'hamming_loss':    hamming_loss(y_true, y_pred),
        'subset_accuracy': accuracy_score(y_true, y_pred),
        'macro_precision': precision_score(y_true, y_pred, average='macro'),
        'macro_recall':    recall_score(y_true, y_pred, average='macro'),
        'macro_f1':        f1_score(y_true, y_pred, average='macro'),
        'micro_f1':        f1_score(y_true, y_pred, average='micro'),
        'samples_f1':      f1_score(y_true, y_pred, average='samples'),
        'jaccard_score':   jaccard_score(y_true, y_pred, average='samples')
    }
    
    # Метрики по отдельным меткам
    if label_columns is not None:
        per_label_metrics = {}
        for i, label in enumerate(label_columns):
            per_label_metrics[label] = {
                'precision': precision_score(y_true[:, i], y_pred[:, i]),
                'recall': recall_score(y_true[:, i], y_pred[:, i]),
                'f1': f1_score(y_true[:, i], y_pred[:, i])
            }
        metrics['per_label'] = per_label_metrics
    
    return metrics

class LabelAttention(nn.Module):
    """
    Description:
        Механизм внимания для учета взаимосвязей между метками в задаче 
        многометочной классификации.
        
    Args:
        hidden_size (int): Размерность скрытого состояния модели
        num_labels (int): Количество меток для классификации
        
    Returns:
        torch.Tensor: Тензор с примененным механизмом внимания
        
    Examples:
        >>> attention = LabelAttention(hidden_size=768, num_labels=10)
        >>> hidden_states = torch.randn(32, 512, 768)  # batch_size=32, seq_len=512
        >>> output = attention(hidden_states)
        >>> output.shape
        torch.Size([32, 10, 768])
    """
    def __init__(self, hidden_size: int, num_labels: int):
        super().__init__()
        
        # Инициализация многоголового механизма внимания
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,    # Размерность эмбеддингов
            num_heads=8,              # Количество голов внимания
            dropout=0.1,              # Вероятность dropout
            batch_first=True          # Формат входных данных [batch, seq, features]
        )
        
        # Инициализация эмбеддингов меток как обучаемых параметров
        self.label_embeddings = nn.Parameter(
            torch.randn(num_labels, hidden_size)
        )
        
    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        """
        Description:
            Применяет механизм внимания к входным скрытым состояниям
        
        Args:
            hidden_states (torch.Tensor): Входной тензор скрытых состояний размерности
                                         (batch_size, sequence_length, hidden_size)
                                         
        Returns:
            torch.Tensor: Выходной тензор после применения механизма внимания
                         (batch_size, num_labels, hidden_size)
        """
        batch_size = hidden_states.size(0)  # Определяем размер batch
        
        # Расширяем embedding меток для каждого элемента в batch
        label_embeddings = self.label_embeddings.unsqueeze(0).expand(
            batch_size, -1, -1
        )  # (batch_size, num_labels, hidden_size)
        
        # Применяем механизм внимания
        attn_output, _ = self.attention(
            query=label_embeddings,  # Метки в качестве запроса (Q)
            key=hidden_states,       # Скрытые состояния в качестве ключей (K)
            value=hidden_states      # Скрытые состояния в качестве значений (V)
        )
        
        return attn_output

class FocalLoss(nn.Module):
    """
    Description:
        Focal Loss для работы с несбалансированными классами, 
        уменьшает влияние легко классифицируемых примеров и 
        фокусируется на сложных примерах.

    Args:
        alpha (float, optional): Весящий коэффициент для контроля вклада редких классов (default: 1.0)
        gamma (float, optional): Параметр фокусировки, увеличивающий вес сложных примеров (default: 2.0)
        reduction (str, optional): Способ агрегации результатов, может быть 'mean', 'sum' или 'none' (default: 'mean')

    Returns:
        torch.Tensor: Значение функции потерь

    Examples:
        >>> criterion = FocalLoss(alpha=1.0, gamma=2.0, reduction='mean')
        >>> inputs = torch.randn(10, requires_grad=True)
        >>> targets = torch.randint(0, 2, (10,), dtype=torch.float)
        >>> loss = criterion(inputs, targets)
        >>> loss.backward()
    """
    def __init__(
        self,
        alpha: float = 1.0,
        gamma: float = 2.0,
        reduction: str = 'mean'
    ):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(
        self,
        inputs: torch.Tensor,
        targets: torch.Tensor
    ) -> torch.Tensor:
        """
        Вычисляет Focal Loss.
        
        Args:
            inputs (torch.Tensor): Входные предсказания (логиты)
            targets (torch.Tensor): Истинные значения (0 или 1)
        
        Returns:
            torch.Tensor: Значение Focal Loss
        """
        # Вычисляем BCE Loss
        bce_loss = F.binary_cross_entropy_with_logits(
            inputs,
            targets,
            reduction='none'
        )
        
        # Вычисляем Focal Loss
        pt = torch.exp(-bce_loss)  # Вероятность предсказанного класса
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        
        # Применяем reduction
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


class MultilabelClassifier(nn.Module):
    """
    Description:
        Улучшенная модель для многометочной классификации текстов.
    
    Args:
        model_name (str): Название предобученной модели
        num_labels (int): Количество меток для классификации
        hidden_dropout_prob (float): Вероятность dropout
        focal_loss_params (Dict): Параметры для Focal Loss
        
    Attributes:
        bert (AutoModelForSequenceClassification): Базовая BERT модель
        label_attention (LabelAttention): Механизм внимания для меток
        intermediate (nn.Sequential): Промежуточные слои
        classifier (nn.Linear): Финальный классификационный слой
        criterion (FocalLoss): Функция потерь
    """
    
    def __init__(
        self,
        model_name: str,
        num_labels: int,
        hidden_dropout_prob: float = 0.2,
        focal_loss_params: Optional[Dict] = None
    ) -> None:
        super().__init__()
        
        # Загружаем базовую модель
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            problem_type="multi_label_classification",
            ignore_mismatched_sizes=True
        )
        
        # Получаем размер скрытого состояния
        hidden_size = self.bert.config.hidden_size
        
        # Инициализируем Label Attention
        self.label_attention = LabelAttention(
            hidden_size=hidden_size,
            num_labels=num_labels
        )
        
        # Промежуточные слои с расширенной архитектурой
        self.intermediate = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.GELU(),
            nn.Dropout(hidden_dropout_prob),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.GELU(),
            nn.Dropout(hidden_dropout_prob)
        )
        
        # Финальный классификационный слой
        self.classifier = nn.Linear(hidden_size // 2, num_labels)
        
        # Инициализируем Focal Loss
        if focal_loss_params is None:
            focal_loss_params = {'alpha': 1.0, 'gamma': 2.0}
        self.criterion = FocalLoss(**focal_loss_params)

    def get_bert_outputs(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Получает выходы из BERT модели.
        """
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Получаем последнее скрытое состояние
        last_hidden_state = outputs.hidden_states[-1]
        
        # Применяем attention к последнему скрытому состоянию
        attention_output = self.label_attention(last_hidden_state)
        
        return last_hidden_state, attention_output

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        labels: Optional[torch.Tensor] = None
    ) -> Any:
        """
        Description:
            Прямой проход через модель.
        
        Args:
            input_ids: Входные ID токенов
            attention_mask: Маска внимания
            labels: Метки (опционально)
            
        Returns:
            Выходы модели с логитами и потерями
        """
        # Получаем выходы BERT и attention
        hidden_states, attention_output = self.get_bert_outputs(
            input_ids,
            attention_mask
        )
        
        # Конкатенируем последнее скрытое состояние и выход attention
        concatenated = torch.cat(
            [hidden_states[:, 0, :], attention_output.mean(dim=1)],
            dim=1
        )
        
        # Пропускаем через промежуточные слои
        intermediate_output = self.intermediate(concatenated)
        
        # Получаем логиты
        logits = self.classifier(intermediate_output)
        
        # Формируем выходной словарь
        outputs = {
            'logits': logits
        }
        
        # Если переданы метки, вычисляем потери
        if labels is not None:
            outputs['loss'] = self.criterion(logits, labels)
        
        return outputs

    def predict(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        threshold: float = 0.5
    ) -> torch.Tensor:
        """
        Получение предсказаний с применением порога.
        """
        outputs = self.forward(input_ids, attention_mask)
        predictions = torch.sigmoid(outputs['logits']) > threshold
        return predictions.float()

def calculate_class_weights(
    labels: np.ndarray,
    beta: float = 0.999,
    min_weight: float = 0.1,
    max_weight: float = 10.0,
    eps: float = 1e-6
) -> torch.Tensor:
    """
    Description:
        Вычисляет сбалансированные веса для классов.
    
    Args:
        labels: Массив меток
        beta: Фактор сглаживания
        min_weight: Минимальный вес
        max_weight: Максимальный вес
    
    Returns:
        torch.Tensor: Тензор весов
    """
    positive_counts = labels.sum(axis=0)
    total_samples = len(labels)
    
    # Защита от нулевых значений
    positive_counts = np.where(positive_counts == 0, eps, positive_counts)
    
    # Расчет эффективного числа примеров
    effective_num = 1.0 - np.power(beta, positive_counts)
    weights = (1.0 - beta) / np.where(effective_num < eps, eps, effective_num)
    
    # Нормализация относительно общего числа примеров
    weights = weights * (total_samples / (weights.sum() + eps))
    
    # Логарифмическое сжатие для уменьшения диспропорций
    weights = np.log(weights + 1.0)
    
    # Масштабирование в заданный диапазон
    weights = (weights - weights.min()) / (weights.max() - weights.min() + eps)
    weights = weights * (max_weight - min_weight) + min_weight
    
    return torch.tensor(weights, dtype=torch.float32)


def train_model(
    model: MultilabelClassifier,
    train_loader: DataLoader,
    val_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    scheduler: torch.optim.lr_scheduler._LRScheduler,
    device: torch.device,
    num_epochs: int,
    patience: int,
    output_dir: Path,
    label_columns: List[str],
    pos_weights: torch.Tensor,
    label_smoothing: float = 0.1
) -> MultilabelClassifier:
    """
    Description:
        Функция обучения модели с логированием и сглаживанием меток.
        
    Args:
        model: Модель для обучения
        train_loader: Загрузчик обучающих данных
        val_loader: Загрузчик валидационных данных
        optimizer: Оптимизатор
        scheduler: Планировщик скорости обучения
        device: Устройство для обучения (CPU/GPU)
        num_epochs: Количество эпох обучения
        patience: Количество эпох для ранней остановки
        output_dir: Директория для сохранения результатов
        label_columns: Названия меток
        pos_weights: Веса для классов
        label_smoothing: Коэффициент сглаживания меток
        
    Returns:
        Обученная модель
    """
    def smooth_labels(
        labels: torch.Tensor,
        smoothing: float,
        eps: float = 1e-6
    ) -> torch.Tensor:
        """
        Description:
            Применяет сглаживание к бинарным меткам для многометочной классификации.
        
        Args:
            labels: Тензор бинарных меток размера (batch_size, num_classes)
            smoothing: Коэффициент сглаживания (0-1)
            eps: Малое число для численной стабильности
        
        Returns:
            torch.Tensor: Тензор сглаженных меток
        
        Raises:
            ValueError: Если smoothing не в диапазоне [0, 1)
        """
        if not 0 <= smoothing < 1:
            raise ValueError("Коэффициент сглаживания должен быть в диапазоне [0, 1)")
        
        if smoothing == 0:
            return labels
            
        with torch.no_grad():
            # Защита от изменения оригинальных меток
            smoothed = labels.clone().float()
            
            # Сглаживание: позитивные метки -> (1-smoothing), негативные -> smoothing
            smoothed = smoothed * (1.0 - smoothing) + (1.0 - smoothed) * smoothing
            
            # Обеспечиваем численную стабильность
            smoothed = torch.clamp(smoothed, min=eps, max=1.0 - eps)
            
            return smoothed
    
    # Инициализация функции потерь с весами
    criterion = nn.BCEWithLogitsLoss(
        pos_weight=pos_weights
    )
    
    # Инициализация переменных для отслеживания процесса обучения
    best_val_loss = float('inf')
    early_stopping_counter = 0
    
    # Словарь для хранения истории обучения
    history = {
        'train_loss': [],
        'val_loss': [],
        'metrics': []
    }
    
    print("\nНачало обучения:")
    print("-" * 50)
    
    # Основной цикл обучения по эпохам
    for epoch in range(num_epochs):
        print(f"\nЭпоха {epoch + 1}/{num_epochs}")
        print("=" * 50)
        
        # ===================== ФАЗА ОБУЧЕНИЯ =====================
        model.train()
        train_loss = 0
        
        print("\nОбучение:")
        for batch in tqdm(train_loader, desc='Прогресс'):
            # Обнуление градиентов
            optimizer.zero_grad()
            
            # Перемещение данных на устройство
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            # Применяем сглаживание к меткам
            smoothed_labels = smooth_labels(labels, label_smoothing)
            
            # Прямой проход
            outputs = model(input_ids, attention_mask)
            # Вычисление функции потерь
            loss = criterion(outputs['logits'], smoothed_labels)
            
            # Обратный проход и оптимизация
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
        
        # Средняя ошибка за эпоху
        avg_train_loss = train_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)
        
        # ===================== ФАЗА ВАЛИДАЦИИ =====================
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        
        print("\nВалидация:")
        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Прогресс'):
                # Перемещение данных на устройство
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                # Получение предсказаний
                outputs = model(input_ids, attention_mask)
                # Используем оригинальные метки для валидации
                loss = criterion(outputs['logits'], labels)
                
                val_loss += loss.item()
                
                # Сбор предсказаний
                preds = torch.sigmoid(outputs['logits']).cpu().numpy()
                all_preds.extend(preds)
                all_labels.extend(labels.cpu().numpy())
        
        # Средняя ошибка валидации
        avg_val_loss = val_loss / len(val_loader)
        history['val_loss'].append(avg_val_loss)
        
        # ===================== ВЫЧИСЛЕНИЕ МЕТРИК =====================
        all_preds = np.array(all_preds) > 0.5
        all_labels = np.array(all_labels)
        
        # Расчет метрик
        metrics = evaluate_multilabel(all_labels, all_preds, label_columns)
        history['metrics'].append(metrics)
        
        # Вывод общих метрик
        print("\nОбщие метрики мультилейбл классификации:")
        print("-" * 50)
        for metric_name, value in metrics.items():
            if metric_name != 'per_label':
                print(f"{metric_name:20s}: {value:.4f}")
        
        # Вывод метрик по меткам
        print("\nМетрики по отдельным меткам:")
        print("-" * 50)
        for label, label_metrics in metrics['per_label'].items():
            print(f"{label:25s}:", end=" ")
            print(", ".join([
                f"{k}: {v:.4f}"
                for k, v in label_metrics.items()
            ]))
        
        # Вывод результатов эпохи
        print("\nРезультаты эпохи:")
        print(f"Средняя ошибка обучения:    {avg_train_loss:.4f}")
        print(f"Средняя ошибка валидации:   {avg_val_loss:.4f}")
        
        # ===================== РАННЯЯ ОСТАНОВКА =====================
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            early_stopping_counter = 0
            
            # Сохранение лучшей модели
            print(f"\nСохранение лучшей модели в {output_dir}/best_model")
            model.bert.save_pretrained(Path(output_dir) / "best_model")
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print("\nРанняя остановка!")
                break
        
        # Обновление learning rate
        scheduler.step()
    
    # Сохранение истории обучения
    history_path = Path(output_dir) / "training_history.json"
    print(f"\nСохранение истории обучения в {history_path}")
    with open(history_path, "w") as f:
        json.dump(history, f)
    
    return model


def main() -> None:
    """
    Description:
        Основная функция для обучения модели многометочной классификации.
    """
    # Конфигурация
    CONFIG = {
        # Базовые параметры в первом приближении
        'model_id': "modernBERT-large-multilingual",          # Идентификатор модели
        'data_path': "data/dataset.csv",                      # Путь к данным
        'output_dir': "modernbert-russian-multilabel-final",  # Директория для вывода результатов
        'max_length': 1024,                                   # Максимальная длина последовательности
        'num_epochs': 5,                                      # Количество эпох для обучения
        'patience': 3,                                        # Параметр терпения для ранней остановки
        'learning_rate': 2e-5,                                # Скорость обучения
        'weight_decay': 0.01,                                 # Коэффициент регуляризации (L2)
        'train_batch_size': 16,                               # Размер батча для обучения
        'eval_batch_size': 32,                                # Размер батча для оценки
    
        # Новые гиперпараметры (эмпирическое приближение)
        'gradient_clip_max_norm': 1.0,                        # Ограничение градиентов для борьбы с взрывом градиентов
        'dropout_rate': 0.1,                                  # Регуляризация через дропаут
        'scheduler_type': 'cosine',                           # Тип расписания: 'linear', 'cosine', 'warmup'
        'warmup_steps': 100,                                  # Шаги разогрева для адаптации скорости обучения
        'accumulation_steps': 2,                              # Аккумуляция градиентов для виртуального батча
        'pos_weight': 'balanced',                             # Веса для дисбаланса классов ('balanced' или массив)
        'threshold': 0.4,                                     # Оптимальный порог классификации (можно подобрать на валидации)
        'use_fp16': True,                                     # Использование смешанной точности для ускорения
        'num_workers': 8,                                     # Параллелизм загрузки данных
        'label_smoothing': 0.1,                               # Сглаживание меток для борьбы с переобучением
        'layerwise_lr_decay': 0.85,                           # Дифференцированный LR по слоям BERT
        'freeze_layers': 6,                                   # Количество замороженных нижних слоев BERT

        # Параметры для весов меток
        'class_weight_beta': 0.9999,                          # Фактор сглаживания для расчета весов классов
        'class_weight_min': 0.5,                              # Минимальный вес класса
        'class_weight_max': 5.0,                              # Максимальный вес класса
        'label_smoothing': 0.1,                               # Сглаживание меток для борьбы с переобучением
    }

    
    
    # ================== ИНИЦИАЛИЗАЦИЯ ==================
    print("\nНачало работы")
    print(f"Модель: {CONFIG['model_id']}")
    print(f"Данные: {CONFIG['data_path']}")
    print(f"Выходная директория: {CONFIG['output_dir']}")
    
    output_dir = Path(CONFIG['output_dir'])
    output_dir.mkdir(parents=True, exist_ok=True)

    # ================== ПОДГОТОВКА ДАННЫХ ==================
    texts, labels, label_columns = load_data(CONFIG['data_path'])
    num_labels = len(label_columns)
    
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        texts, labels, test_size=0.2, random_state=42
    )
    
    # ================== РАСЧЕТ ВЕСОВ КЛАССОВ ==================
    print("\nРасчет весов для классов:")
    pos_weights = calculate_class_weights(
        train_labels,
        beta=CONFIG['class_weight_beta'],
        min_weight=CONFIG['class_weight_min'],
        max_weight=CONFIG['class_weight_max']
    )
    
    # Валидация весов
    assert pos_weights.shape[0] == len(label_columns), "Ошибка: количество весов не соответствует числу меток"
    
    print("\nПолученные веса классов (нормализованные):")
    normalized_weights = pos_weights / pos_weights.sum()
    for label, weight, norm_weight in zip(label_columns, pos_weights, normalized_weights):
        print(f"{label:25s}: raw={weight:.2f}, norm={norm_weight:.4f}")

    # ================== ИНИЦИАЛИЗАЦИЯ МОДЕЛИ ==================
    tokenizer = AutoTokenizer.from_pretrained(CONFIG['model_id'])
    model = MultilabelClassifier(CONFIG['model_id'], num_labels)

    # print(model)

    # ================== ПОДГОТОВКА ДАТАЛОАДЕРОВ ==================
    train_dataset = MultiLabelTextDataset(
        train_texts, train_labels, tokenizer, CONFIG['max_length']
    )
    val_dataset = MultiLabelTextDataset(
        val_texts, val_labels, tokenizer, CONFIG['max_length']
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['train_batch_size'],
        shuffle=True,
        num_workers=CONFIG['num_workers']
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG['eval_batch_size'],
        num_workers=CONFIG['num_workers']
    )

    # ================== НАСТРОЙКА ОБУЧЕНИЯ ==================
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nИспользуемое устройство: {device}")
    
    model.to(device)
    pos_weights = pos_weights.to(device)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=CONFIG['learning_rate'],
        weight_decay=CONFIG['weight_decay']
    )
    
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=1.0,
        end_factor=0.0,
        total_iters=CONFIG['num_epochs']
    )

    # ================== ОБУЧЕНИЕ МОДЕЛИ ==================
    model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        num_epochs=CONFIG['num_epochs'],
        patience=CONFIG['patience'],
        output_dir=output_dir,
        label_columns=label_columns,
        pos_weights=pos_weights,
        label_smoothing=CONFIG['label_smoothing']
    )

    # ================== СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ==================
    print(f"\nСохранение финальной модели в {output_dir}/final_model")
    model.bert.save_pretrained(output_dir / "final_model")
    tokenizer.save_pretrained(output_dir / "final_model")
    
    label_columns_path = output_dir / "label_columns.json"
    with open(label_columns_path, "w") as f:
        json.dump(label_columns, f)
    
    print("\nОбучение завершено!")

/home/username/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
main()


Начало работы
Модель: modernBERT-large-multilingual
Данные: data/dataset.csv
Выходная директория: modernbert-russian-multilabel-final

Загрузка данных из data/dataset.csv...
Загружено 35303 записей с 19 метками

Расчет весов для классов:

Полученные веса классов (нормализованные):
оффлайн_преступления     : raw=3.07, norm=0.0506
онлайн_преступления      : raw=5.00, norm=0.0825
наркотики                : raw=0.67, norm=0.0110
азартные_игры            : raw=2.67, norm=0.0440
порнография              : raw=2.82, norm=0.0465
проституция              : raw=4.22, norm=0.0697
рабство                  : raw=4.90, norm=0.0808
самоубийство             : raw=3.95, norm=0.0651
терроризм                : raw=4.26, norm=0.0702
оружие                   : raw=2.42, norm=0.0400
бодишейминг              : raw=4.48, norm=0.0739
хейлтшейминг             : raw=3.32, norm=0.0547
политика                 : raw=2.56, norm=0.0422
расизм                   : raw=3.62, norm=0.0597
религия                  : raw=

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at modernBERT-large-multilingual and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([19]) in the model instantiated
- classifier.weight: found shape torch.Size([3, 1024]) in the checkpoint and torch.Size([19, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Используемое устройство: cuda

Начало обучения:
--------------------------------------------------

Эпоха 1/5

Обучение:


Прогресс: 100%|██████████| 1766/1766 [32:18<00:00,  1.10s/it]



Валидация:


...


Общие метрики мультилейбл классификации:
--------------------------------------------------
hamming_loss        : 0.0240
subset_accuracy     : 0.7141
macro_precision     : 0.7440
macro_recall        : 0.8459
macro_f1            : 0.7882
micro_f1            : 0.7923
samples_f1          : 0.7578
jaccard_score       : 0.7335

Метрики по отдельным меткам:
--------------------------------------------------
оффлайн_преступления     : precision: 0.5393, recall: 0.6749, f1: 0.5995
онлайн_преступления      : precision: 0.6545, recall: 0.7970, f1: 0.7188
наркотики                : precision: 0.9729, recall: 0.9058, f1: 0.9382
азартные_игры            : precision: 0.8701, recall: 0.9259, f1: 0.8971
порнография              : precision: 0.5850, recall: 0.7786, f1: 0.6680
проституция              : precision: 0.7639, recall: 0.9357, f1: 0.8412
рабство                  : precision: 0.7763, recall: 0.8939, f1: 0.8310
самоубийство             : precision: 0.8958, recall: 0.9348, f1: 0.9149
терроризм 

Прогресс: 100%|██████████| 1766/1766 [32:17<00:00,  1.10s/it]



Валидация:


Прогресс: 100%|██████████| 221/221 [02:50<00:00,  1.30it/s]
/home/username/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/username/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Общие метрики мультилейбл классификации:
--------------------------------------------------
hamming_loss        : 0.0212
subset_accuracy     : 0.7373
macro_precision     : 0.7689
macro_recall        : 0.8364
macro_f1            : 0.7985
micro_f1            : 0.8099
samples_f1          : 0.7598
jaccard_score       : 0.7383

Метрики по отдельным меткам:
--------------------------------------------------
оффлайн_преступления     : precision: 0.6081, recall: 0.6530, f1: 0.6298
онлайн_преступления      : precision: 0.6627, recall: 0.8168, f1: 0.7317
наркотики                : precision: 0.9677, recall: 0.9125, f1: 0.9392
азартные_игры            : precision: 0.8744, recall: 0.9284, f1: 0.9006
порнография              : precision: 0.6733, recall: 0.7262, f1: 0.6987
проституция              : precision: 0.7872, recall: 0.9357, f1: 0.8550
рабство                  : precision: 0.6525, recall: 0.9293, f1: 0.7667
самоубийство             : precision: 0.9117, recall: 0.9348, f1: 0.9231
терроризм 

# Итог второго раунда обучения с применением весов классов ⚖️

В этом раунде для борьбы с дисбалансом классов были рассчитаны и применены веса для каждого класса. Ниже приведён краткий разбор полученных результатов по сравнению с вариантом без взвешивания.

---

### 🔍 Основные наблюдения:

- **Изменения в функциях потерь и обучении:**
  - При использовании весов средние значения ошибки (train loss и val loss) оказались значительно выше (около 0.68–0.34) по сравнению с ранним раундом без весов (где ошибки снижались до ~0.0017). Это связано с масштабированием функции потерь при применении весов, что отражает иной баланс вкладов редких и частых классов.

- **Общие метрики:**
  - **Hamming Loss:**  
    Немного увеличилась с ~0.0172 до 0.0212 👀 — небольшое ухудшение.
  - **Subset Accuracy:**  
    Понизилась с 76.1% до 73.7% 📉.
  - **Macro Precision:**  
    Значительно снизилась (с ~89.95% до 76.89%) 🔻, что говорит о росте числа ложноположительных предсказаний для некоторых классов.
  - **Macro Recall:**  
    Улучшилась (с 73.47% до 83.64%) 🔺 — модель стала ловить больше истинных позитивов.
  - **Macro F1:**  
    Осталась примерно на прежнем уровне (80.31% против 79.85%), что указывает на компромисс между точностью и полнотой.
  - **Micro F1:**  
    Незначительно снизилась (с 82.49% до 80.99%).
  - **Samples F1 и Jaccard Score:**  
    Незначительно улучшились (Samples F1 выросла с 72.06% до 75.98%, а Jaccard — с 70.82% до 73.83%) 😊.

- **Метрики по отдельным меткам:**
  - Для некоторых классов, например, **«онлайн_преступления»** и **«порнография»**, наблюдается снижение precision, но заметное повышение recall.  
  - Такое смещение говорит о том, что модель стала более чувствительной к редким примерам, но за счёт этого генерирует больше ложных срабатываний.

---

### 📊 Вывод:

Применение весов для классов позволило **повысить recall и улучшить метрику Jaccard**, что особенно важно, если требуется минимизировать пропуски по редким классам. Однако это произошло за счёт снижения precision и subset accuracy — модель стала генерировать больше ложноположительных срабатываний. Итоговый **Macro F1** остался практически неизменным, что свидетельствует о том, что баланс между точностью и полнотой сместился, но суммарное качество предсказаний осталось на сопоставимом уровне.

**Заключение:**  
Если приоритетом является снижение количества пропусков (увеличение recall) и улучшение метрики Jaccard, то применение весов для классов даёт положительный эффект. В противном случае (если критична высокая точность) может потребоваться дополнительная настройка гиперпараметров или корректировка схемы взвешивания.

Предлагаю Вам дообагатить датасет что бы сбалансировать классы и прогнать на тетрадке первого раунда, думаю результат должен быть весьма хороший.

🚀 Продолжаем экспериментировать для оптимизации компромисса между precision и recall!


---